In [1]:
import java.io.File
import ammonite.api.Classpath

import java.io.File
import ammonite.api.Classpath

In [2]:
val f:File = new File(System.getProperty("user.home") + "/git/dl4j-examples/dl4j-examples/target/dependency")
val n = f.listFiles.size
var i = 0
f.listFiles.map(_.toString).filter(_.endsWith("jar")).par.foreach((f: String) => { 
    classpath.addPath(f.toString)
    i += 1
    if (i%10==0) print(f"${i.toDouble / n * 100}%4.2f%%...")
})

3.42%...6.85%...10.27%...13.70%...17.12%...20.55%...23.97%...27.40%...30.82%...34.25%...37.67%...41.10%...44.52%...47.95%...51.37%...54.79%...58.22%...61.64%...65.07%...68.49%...71.92%...75.34%...78.77%...82.19%...85.62%...89.04%...92.47%...95.89%...99.32%...

f: java.io.File = /Users/behnamrasoolian/git/dl4j-examples/dl4j-examples/target/dependency
n: Int = 292
i: Int = 292

In [3]:
//myBlock(classpath)

In [4]:
def runInThread[T](a: T)(block: T => Unit) {
new Thread {
    override def run() { block(a) }
    }.start()
}

defined function runInThread

In [5]:
//runInThread(classpath)(myBlock)

In [6]:
runInThread(System.out)(_.println("mardas"))

mardas


In [33]:
import org.bytedeco.javacpp.opencv_core._
import org.bytedeco.javacpp.opencv_core

import org.bytedeco.javacpp.opencv_core._
import org.bytedeco.javacpp.opencv_core

In [9]:
import org.nd4s.Implicits._
import org.datavec.image.loader.NativeImageLoader

import org.nd4s.Implicits._
import org.datavec.image.loader.NativeImageLoader

In [10]:
val loader = new NativeImageLoader(100, 200)

loader: org.datavec.image.loader.NativeImageLoader = org.datavec.image.loader.NativeImageLoader@5841af40

In [11]:
val image = loader.asMatrix(new File("/Users/behnamrasoolian/Dropbox/notebook/sesek/figures/fig_violing_plot.png"))

23:39:06.241 [pool-4-thread-3] INFO org.nd4j.linalg.factory.Nd4jBackend - Loaded [CpuBackend] backend
23:39:06.338 [pool-4-thread-3] WARN org.reflections.Reflections - given scan urls are empty. set urls in the configuration
23:39:07.979 [pool-4-thread-3] INFO org.nd4j.nativeblas.NativeOpsHolder - Number of threads used for NativeOps: 2
23:39:08.396 [pool-4-thread-3] INFO org.nd4j.nativeblas.Nd4jBlas - Number of threads used for BLAS: 2
23:39:08.508 [pool-4-thread-3] INFO org.nd4j.linalg.api.ops.executioner.DefaultOpExecutioner - Backend used: [CPU]; OS: [Mac OS X]
23:39:08.509 [pool-4-thread-3] INFO org.nd4j.linalg.api.ops.executioner.DefaultOpExecutioner - Cores: [4]; Memory: [0.9GB];
23:39:08.509 [pool-4-thread-3] INFO org.nd4j.linalg.api.ops.executioner.DefaultOpExecutioner - Blas vendor: [OPENBLAS]
23:39:08.599 [pool-4-thread-3] DEBUG org.reflections.Reflections - going to scan these urls:
jar:file:/Users/behnamrasoolian/git/dl4j-examples/dl4j-examples/target/dependency/nd4j-nativ

image: org.nd4j.linalg.api.ndarray.INDArray = [[[[         0,         0,         0  ...         0         0,         0], 
   [         0,         0,         0  ...         0         0,         0], 
   [         0,         0,         0  ...         0         0,         0], 
    ..., 
   [         0,         0,         0  ...         0         0,         0], 
   [         0,         0,         0  ...         0         0,         0], 
   [         0,         0,         0  ...         0         0,         0]], 

  [[         0,         0,         0  ...         0         0,         0], 
   [         0,         0,         0  ...         0         0,         0], 
   [         0,         0,         0  ...         0         0,         0], 
    ..., 
   [         0,         0,         0  ...         0         0,         0], 
   [         0,         0,         0  ...         0         0,         0], 
   [         0,         0,         0  ...         0         0,         0]], 

  [[         0,   

In [30]:
image(0, 1, --->).shape

res29: Array[Int] = Array(100, 200)

In [32]:
val indices = new Mat()
val mat = loader.asMat(image)

indices: org.bytedeco.javacpp.opencv_core.Mat = org.bytedeco.javacpp.opencv_core$Mat[width=0,height=0,depth=8,channels=1]
mat: org.bytedeco.javacpp.opencv_core.Mat = org.bytedeco.javacpp.opencv_core$Mat[width=200,height=100,depth=32,channels=3]

In [ ]:
org.bytedeco.javacpp.opencv_core